# Experiments:
The experiments are done on Traffic Signs classification case study. Signs classification is a component of autonomous
vehicles and therefore its resistance to adversarial attacks is highly crucial. We talk about that much more in the
project report.

The dataset is GTSRB (German Traffic Sign Recognition).
I downloaded the dataset from: https://github.com/tomlawrenceuk/GTSRB-Dataloader

In short the experiments are:
1) Apply PGD and FGSM attacks on a spatial convolutional neural network.
2) Comparing robustness of adversarial training using FGSM versus using PGD.
3) Universality of PGD attack - we show that using PGD adversarial training we cover any first order attack (we show
   resistance to some known adversarial attacks - not a theoretical proof)
4) Capacity and Adversarial Robustness (see more details in Experiment 4)


In [1]:
# import libraries:

import torch
import attacks
import configs
import datasets
import dls
import helper
import models
import trainer
import os
import shutil


Initialization:
    - export configs
    - validate paths
    - set random seed
    - configure device (GPU / CPU)
    - load datasets
    - create hyperparameters generators

In [2]:
# configs
experiment_configs = configs.TrafficSigns_experiments_configs
experiment_hps_sets = configs.TrafficSigns_experiments_hps
show_test_successful_attacks_plots = configs.show_test_successful_attacks_plots
save_test_successful_attacks_plots = configs.show_test_successful_attacks_plots

# paths existence validation and initialization
assert os.path.exists(configs.data_root_dir), "The dataset should be in ./data/GTSRB"
assert os.path.exists(os.path.join(configs.data_root_dir, "GTSRB")), "The dataset should be in ./data/GTSRB"
if not os.path.exists(configs.results_folder):
    os.mkdir(configs.results_folder)
if os.path.exists(configs.plots_folder):
    shutil.rmtree(configs.plots_folder)
    os.mkdir(configs.plots_folder)
if os.path.exists(configs.logger_path):
    os.remove(configs.logger_path)

# seed
if configs.seed is not None:
    # np.random.seed(configs.seed)
    torch.manual_seed(configs.seed)

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# get datasets
transform = experiment_configs["data_transform"]
_training_dataset = datasets.GTSRB(root_dir=configs.data_root_dir, train=True, transform=transform)
_testing_dataset = datasets.GTSRB(root_dir=configs.data_root_dir, train=False, transform=transform)

# create hyperparameters generators
net_training_hps_gen = helper.GridSearch(experiment_hps_sets["nets_training"])
fgsm_attack_hps_gen = helper.GridSearch(experiment_hps_sets["FGSM"])
pgd_attack_hps_gen = helper.GridSearch(experiment_hps_sets["PGD"])

# loss and general training componenets:
_loss_fn = experiment_configs["loss_function"]
stop_criteria = experiment_configs["stopping_criteria"]
epochs = trainer.Epochs(stop_criteria)

## Experiment 1: PGD and FGSM attacks in practice.

In this experiment we will attack a network using PGD and FGSM attacks.
The experiment illustrates that PGD and FGSM attacks actually works on GTSRB dataset.
We train and attack a Spatial Transformer Network which is invariant to geometrical
transformations (rotations and scaling).


In [3]:
def experiment_1_func(net, _loss_fn, _training_dataset, _testing_dataset, epochs,
                      net_name="", train_attack=None, load_checkpoint=False, save_checkpoint=True):
    if load_checkpoint:
        checkpoint_path = os.path.join(configs.checkpoints_folder, "{}.pt".format(net_name))
        checkpoint = torch.load(checkpoint_path)
        net.load_state_dict(checkpoint["trained_net"])
        net_hp = checkpoint["net_hp"]
        fgsm_hp = checkpoint["fgsm_hp"]
        pgd_hp = checkpoint["pgd_hp"]
        resistance_results = checkpoint["resistance_results"]

    else:
        # apply hyperparameters-search to get trained network
        net, net_hp, net_acc = helper.full_train_of_nn_with_hps(net, _loss_fn, _training_dataset,
                                                                net_training_hps_gen,
                                                                epochs, device=device, train_attack=train_attack)
        net.eval()

        # attack net (trained) using FGSM:
        _, fgsm_hp, fgsm_score = helper.full_attack_of_trained_nn_with_hps(net, _loss_fn, _training_dataset,
                                                                           fgsm_attack_hps_gen, net_hp,
                                                                           attacks.FGSM,
                                                                           device=device,
                                                                           plot_successful_attacks=False)

        # attack net (trained) using PGD:
        _, pgd_hp, pgd_score = helper.full_attack_of_trained_nn_with_hps(net, _loss_fn, _training_dataset,
                                                                         pgd_attack_hps_gen, net_hp, attacks.PGD,
                                                                         device=device,
                                                                         plot_successful_attacks=False)

        # measure attacks on test (holdout)
        resistance_results = helper.measure_resistance_on_test(net, _loss_fn, _testing_dataset,
                                                               [(attacks.FGSM, fgsm_hp),
                                                                (attacks.PGD, pgd_hp)],
                                                               plot_successful_attacks=show_test_successful_attacks_plots,
                                                               device=device)

    test_acc = resistance_results["test_acc"]  # the accuracy without applying any attack
    fgsm_res = resistance_results["%fgsm"]
    pgd_res = resistance_results["%pgd"]

    # print scores:
    print("TEST SCORES of {}:".format(net_name))
    print("accuracy on test:            {}".format(test_acc))
    print("%FGSM successful attacks:    {}".format(fgsm_res))
    print("%PGD successful attacks:     {}\n".format(pgd_res))

    # save checkpoint
    res_dict = {
        "trained_net": net,
        "net_hp": net_hp,
        "fgsm_hp": fgsm_hp,
        "pgd_hp": pgd_hp,
        "resistance_results": resistance_results
    }

    if save_checkpoint:
        to_save_res_dict = res_dict
        to_save_res_dict["trained_net"] = net.state_dict()
        checkpoint_path = os.path.join(configs.checkpoints_folder, "{}.pt".format(net_name))
        torch.save(to_save_res_dict, checkpoint_path)

    return res_dict


# define network
net_arch = models.TrafficSignNet().to(device)
# apply experiment 1
exp1_res_dict = experiment_1_func(net_arch, _loss_fn, _training_dataset, _testing_dataset, epochs,
                                      net_name="Spatial Transformer Network(STN)")
# save results
original_trained_net = exp1_res_dict["trained_net"]
net_hp = exp1_res_dict["net_hp"]
fgsm_hp = exp1_res_dict["fgsm_hp"]
pgd_hp = exp1_res_dict["pgd_hp"]


TypeError: 'builtin_function_or_method' object is not iterable

## Experiment 2: Comparing defensing using FGSM versus using PGD
In this experiment we will build 2 robust networks to the same task as the paper suggest but one will be resistant to FGSM attack
and the second to PGD attack.

Then we will examine the following results of the paper:

1. We show that network 1 (i.e. trained to be resistant to FGSM attack) is resistant to FGSM but not to PGD. Therefore we get that:
 - Resistancy FGSM attack does not implies PGD resistency.
 - FGSM attack is not universal.
2. We show that network 2 (i.e. trained to be resistant to PGD attack) is resistant to both PGD and FGSM.

   In the next experiment we show that network 2 is resistant to some other attacks and this examine the universality of PGD (i.e. that resistancy to
   PGD implies resistancy to any other first order attack).

In [ ]:
# epochs = trainer.Epochs(trainer.ConstantStopping(25))
epochs.restart()
fgsm_robust_net = models.TrafficSignNet().to(device)
fgsm_attack = attacks.FGSM(fgsm_robust_net, _loss_fn, fgsm_hp)
experiment_1_func(fgsm_robust_net, _loss_fn, _training_dataset, _testing_dataset, epochs,
                  net_name="robust net built using FGSM", train_attack=fgsm_attack)

epochs.restart()
pgd_robust_net = models.TrafficSignNet().to(device)
pgd_attack = attacks.PGD(pgd_robust_net, _loss_fn, pgd_hp)
experiment_1_func(pgd_robust_net, _loss_fn, _training_dataset, _testing_dataset, epochs,
                      net_name="robust net built using PGD", train_attack=pgd_attack)


## Experiment 3: Universality of PGD attack
In this experiment we will attack the network that is trained to be resistant to PGD attack with different attacks
with different parameters and show that it is also resistant to those attacks.

Note that we already saw in experiment 2 that this network is resistant to FGSM attack with the specified parameters.

In [ ]:
attacks_lst = [
    (attacks.FGSM, {"epsilon": 0.001}),
    (attacks.FGSM, {"epsilon": 0.005}),
    (attacks.PGD, {"epsilon": 0.4, "steps": 60, "alpha": 0.001}),
    (attacks.PGD, {"epsilon": 0.2, "steps": 20, "alpha": 0.005}),
    (attacks.PGD, {"epsilon": 0.3, "steps": 40, "alpha": 0.005}),
    (attacks.PGD, {"epsilon": 0.2, "steps": 40, "alpha": 0.01}),
    (attacks.MomentumFGSM, {"epsilon": 0.2, "steps": 40, "alpha": 0.01, "momentum":0.9}),
    (attacks.MomentumFGSM, {"epsilon": 0.4, "steps": 30, "alpha": 0.001, "momentum":0.95}),
]
pgd_resistance_results = helper.measure_resistance_on_test(pgd_robust_net, _loss_fn, _testing_dataset, attacks_lst,
                                                           plots_title="robust net built using PGD")


## Experiment 4: Capacity and Adversarial Robustness
In this experiment we will examine the following statements:

1. Capacity alone helps: High capacity models are more robust to adversarial attacks then low capacity models.
2. Weak models may fail to learn non-trivial classifiers: We show that we can't build (using the paper method) a robust model
   when the model is with low capacity. Specifically we will show that after training with the paper method to build a robust
   model we get an extremely underfitted model.
3. The value of the saddle point problem decreases as we increase the capacity
4. More capacity and stronger adversaries decrease transferability: we take transferred adversarial inputs and show that their
   gradients correlation with the source becomes less significant. More details on that experiment can be found in the project file.



Technical Details:
To create the increased capacity networks we use create_conv_nn from models.py. A description on the parameters is in
ConvNN class. In short, channels_lst specifies the number of channels at each layer and extras_blocks_components
specifies the components on each block of the network (e.g. dropout, maxpool). #FC_Layers is the number of layers that
follows the convolutional layers. There are limitations on some of the parameters - 2 <= len(channels_lst) <= 5
and #FC_Layers > 0. CNN_out_channels for applying 1x1 conv layer with CNN_out_channels output channels - None for
ignoring this feature. in_wh is width and height of the pictures. out_size is the number of classes. For more details
see ConvNN class description.

In [ ]:
inc_capacity_nets = []
base_net_params = {
    "channels_lst": [3, 20, 40],  # the first element is number of input channels
    "extras_blocks_components": [],  # ["dropout"],
    # "p_dropout": 0.1,
    "activation": torch.nn.LeakyReLU,
    "out_size": 43,
    "in_wh": 32,
    "#FC_Layers": 2,
    "CNN_out_channels": 30  # apply 1x1 conv layer to achieve that - to control mem. None to not use.
}
for i in range(1, 9):
    base_net_params["channels_lst"] = [3, 10 * i, 20 * i]
    base_net_params["#FC_Layers"] = 1 + i // 2
    base_net_params["CNN_out_channels"] = i * 5
    cap_net = models.create_conv_nn(base_net_params)
    inc_capacity_nets.append(cap_net)
for i, net in enumerate(inc_capacity_nets):
    net = net.to(device)
    epochs.restart()
    experiment_1_func(net, _loss_fn, _training_dataset, _testing_dataset, epochs,
                        net_name="capacity_{}".format(i))

# visualize and stufff........